In [ ]:
dataset = 'ag-vampir-002'
metadata_path = "../../results/config/metadata.qcpass.tsv"
cohort_cols = 'location'
bed_path = "../../config/ag-vampir.bed"
vcf_path = "../../results/vcfs/targets/ag-vampir-002.annot.vcf"
wkdir = "../.."

## Generate metadata from sample sheet

In [ ]:
import pandas as pd

def load_metadata(metadata_path, from_sample_sheet=False):
    
    if not from_sample_sheet:
    # load panel metadata
        if metadata_path.endswith(".xlsx"):
            metadata = pd.read_excel(metadata_path, engine="openpyxl")
        elif metadata_path.endswith(".tsv"):
            metadata = pd.read_csv(metadata_path, sep="\t")
        elif metadata_path.endswith(".csv"):
            metadata = pd.read_csv(metadata_path, sep=",")
        else:
            raise ValueError("Metadata file must be .xlsx or .csv")
    else:
        metadata = metadata_from_sample_sheet(metadata_path)
        
    return metadata

In [ ]:
import os
import pandas as pd
import re

def metadata_from_sample_sheet(input_file):
    with open(input_file, 'r') as f:
        content = f.read()
    
    data_section = re.search(r'\[(BCLConvert_)?Data\],+\n(.*?)($|\[)', content, re.DOTALL)
    if not data_section:
        raise ValueError("Could not find [Data] or [BCLConvert_Data] section in the sample sheet")
    
    data_content = data_section.group(2)
    lines = data_content.strip().split('\n')

    df = pd.DataFrame([line.split(',') for line in lines[1:]], columns=lines[0].split(','))
    df = df.rename(columns={'sample_ID': 'sample_id'})
    
    # Split well column into well_letter and well_number
    df['well_letter'] = df['well'].str[0]
    df['well_number'] = df['well'].str[1:].astype(str)

    return df.loc[['sample_id', 'sample_name', 'index', 'index2', 'plate_name', 'taxon', 'location', 'country', 'latitude', 'longitude', 'well_letter', 'well_number']]
    
# Example usage
input_file = "../../resources/14_02_2024_MiSeq_output/SampleSheet.csv"  # Replace with your input file path
output_file = "results/config/metadata.tsv"

df = metadata_from_sample_sheet(input_file)
df

,sample_id,sample_name,index,index2,plate_name,taxon,location,country,latitude,longitude,,,,well_letter,well_number
0,GH_01,GH_01,ATCACGTT,CCTATCCT,3,,Obuasi,Ghana,,,,,,A,1
1,GH_02,GH_02,CGATGTTT,CCTATCCT,3,,Obuasi,Ghana,,,,,,A,2
2,GH_03,GH_03,TTAGGCAT,CCTATCCT,3,,Obuasi,Ghana,,,,,,A,3
3,GH_04,GH_04,TGACCACT,CCTATCCT,3,,Obuasi,Ghana,,,,,,A,4
4,GH_05,GH_05,ACAGTGGT,CCTATCCT,3,,Obuasi,Ghana,,,,,,A,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
547,Siaya_Delta_Alive_127,Siaya_Delta_Alive_127,TGCGTGAA,CAGGACGT,7,gambiae,Siaya,Kenya,,,,,,H,5
548,Siaya_Delta_Alive_128,Siaya_Delta_Alive_128,GGTGAGTT,CAGGACGT,7,gambiae,Siaya,Kenya,,,,,,H,6
549,Siaya_Delta_Alive_129,Siaya_Delta_Alive_129,GATCTCTT,CAGGACGT,7,gambiae,Siaya,Kenya,,,,,,H,7
550,Siaya_Delta_Alive_130,Siaya_Delta_Alive_130,GTGTCCTT,CAGGACGT,7,gambiae,Siaya,Kenya,,,,,,H,8


In [11]:
df.columns

Index(['sample_id', 'sample_name', 'index', 'index2', 'plate_name', 'taxon',
       'location', 'country', 'latitude', 'longitude', '', '', '',
       'well_letter', 'well_number'],
      dtype='object')